## This notebook explores 4 different small datasets that were kicked back from Lex's R code. 
### 4 csv files were used in this notebook, 7 records back from the duplicates and NA remarks, 1 record back after fixing the IDENT, 1 record has Lat/long from cedar, and 3 records back from duplicates REMARKS.
### The output of this notebook is:
#### 1. 3 + 1 +1 Lat/Long calculated for FAA Airport Identifier
#### 2. 1  Lat/Long calculated for NAVAID
#### 3. 1 + 1 Lat/Long calculated in Cedar
#### 4. 1  Lat/Long calculated for city name
#### 5. 2+ 1 back to Lex

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import geopy
from geopy.distance import geodesic

In [2]:
# Read in csv file with 7 records initially were removed from the original dataset because the REMARKS field is duplicated
nonstd = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/Olga - Additional_Non-Standard_Records_7.csv')
nonstd

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,LONGITUDE,LATITUDE,UASLOCATION,DATASET
0,"Reported a white quad copter at 1,000 FT 4 N o...",5/9/2019,5:25,2019-05-09T05:25Z,LIMO42,DEN,UAS,NaN,OXR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 NM N of NTD,AIRPORT
1,PIC reported a white uas pass 500 feet belowwh...,6/10/2019,1:30,2019-06-10T01:30Z,N523GC,DEN,UAS,NaN,ABE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9 NM E of ETX,NAVAID
2,LUF airspace manager reported to ZAB aircraft ...,12/14/2020,23:04,2020-12-14T23:04Z,ROOK1,DEN,UAS,NOTIFICATION-NONE,ZAB,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10 NM NW of TANKZ,DESIGNATED POINT
3,LUF airspace manager reported to ZAB aircraft ...,12/14/2020,23:55,2020-12-14T23:55Z,DEADLY1,DEN,UAS,NOTIFICATION-NONE,ZAB,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13 NM SE of BUGGS,DESIGNATED POINT
4,Multiple aircraft observed multiple uas off th...,12/25/2020,3:20,2020-12-25T03:20Z,AIP1813,DEN,UAS,NaN,D01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24 NM SW of DEN,AIRPORT
5,NaN,7/18/2021,20:30,2021-07-18T20:30Z,LEGCY11,ATO,UAS,NaN,GSO,7.509909e+08,...,120.0,NaN,79.5702,35.5443,"LEGCY11, 2/V22 GSO overflight reported to RDU ...","Facility summary, weather and falcon bookmark ...",NaN,NaN,NaN,LAT/LONG
6,NaN,7/26/2021,16:08,2021-07-26T16:08Z,EDV4756,ATO,UAS,NaN,AZO,1.052150e+09,...,350.0,1.0,NaN,NaN,"EDV4756 (climbing through 4,300') reported a b...",QA reviewed.,NaN,NaN,2 NM NE of AZO,AIRPORT - CEDAR


In [3]:
# Find records that reference airports
nonstd_air = nonstd[(nonstd['DATASET'] == 'AIRPORT') | (nonstd['DATASET'] == 'AIRPORT - CEDAR')]
nonstd_air

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,LONGITUDE,LATITUDE,UASLOCATION,DATASET
0,"Reported a white quad copter at 1,000 FT 4 N o...",5/9/2019,5:25,2019-05-09T05:25Z,LIMO42,DEN,UAS,NaN,OXR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 NM N of NTD,AIRPORT
4,Multiple aircraft observed multiple uas off th...,12/25/2020,3:20,2020-12-25T03:20Z,AIP1813,DEN,UAS,NaN,D01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24 NM SW of DEN,AIRPORT
6,NaN,7/26/2021,16:08,2021-07-26T16:08Z,EDV4756,ATO,UAS,NaN,AZO,1.052150e+09,...,350.0,1.0,NaN,NaN,"EDV4756 (climbing through 4,300') reported a b...",QA reviewed.,NaN,NaN,2 NM NE of AZO,AIRPORT - CEDAR


In [4]:
# Extract IDENTs
uas_loc = nonstd_air['UASLOCATION'].tolist()
uas_airport = []

for i in range(len(uas_loc)):
    airport = uas_loc[i].split(' ')[-1]
    
    if len(airport) <= 4:
        uas_airport.append(airport)
    else:
        trim_air = airport[-3:]
        uas_airport.append(trim_air)
        
uas_airport_df = pd.DataFrame()
uas_airport_df['IDENT'] = uas_airport
uas_airport_df

,IDENT
0,NTD
1,DEN
2,AZO


In [5]:
# Read in airports dataset
airports = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/airports_cleaned.csv')

In [6]:
# Merge idents dataframe with airports dataset
uas_air_loc= pd.merge(uas_airport_df, airports, on='IDENT', how='left')
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,NTD,Point Mugu NAS (Naval Base Ventura Co),34.119274,-119.119585,KNTD
1,DEN,Denver Intl,39.861667,-104.673167,KDEN
2,AZO,Kalamazoo/Battle Creek Intl,42.234389,-85.551556,KAZO


In [7]:
# Append needed columns
add_col = nonstd_air['REMARKS'].tolist()
uas_air_loc['REMARKS'] = add_col
add_col1 = nonstd_air['UASLOCATION'].tolist()
uas_air_loc['UASLOCATION'] = add_col1
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,REMARKS,UASLOCATION
0,NTD,Point Mugu NAS (Naval Base Ventura Co),34.119274,-119.119585,KNTD,"Reported a white quad copter at 1,000 FT 4 N o...",4 NM N of NTD
1,DEN,Denver Intl,39.861667,-104.673167,KDEN,Multiple aircraft observed multiple uas off th...,24 NM SW of DEN
2,AZO,Kalamazoo/Battle Creek Intl,42.234389,-85.551556,KAZO,NaN,2 NM NE of AZO


In [8]:
# Create dictionary for bearing abbreviations with their degrees
bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
}

# Separate the distances and bearings into two columns
uas_loc = uas_air_loc['UASLOCATION']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(distance_nm[0:])
    bearings.append(bearing_abrev)

# Create DataFrame to store UAS Location Information
new_df = pd.DataFrame(uas_air_loc['REMARKS'])
new_df['UAS_LOC'] = uas_air_loc['UASLOCATION']
new_df['IDENT'] = uas_air_loc['IDENT']
new_df['Distance'] = distances
new_df['Bearing'] = bearings

new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing
0,"Reported a white quad copter at 1,000 FT 4 N o...",4 NM N of NTD,NTD,4,N
1,Multiple aircraft observed multiple uas off th...,24 NM SW of DEN,DEN,24,SW
2,NaN,2 NM NE of AZO,AZO,2,NE


In [13]:
# Convert distances from NM to KM and bearings to degrees
new_df['Distance'] = pd.to_numeric(new_df['Distance'],errors = 'coerce')
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

bearDegree = pd.DataFrame(new_df['Bearing'])
bearDegree = bearDegree.replace({'Bearing': bearing_deg})

new_df['Distance_Kilometers'] = dist_kilo
new_df['Bearing_Degrees'] = bearDegree
new_df['Airport_Latitude'] = uas_air_loc['LATITUDE']
new_df['Airport_Longitude'] = uas_air_loc['LONGITUDE']
new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,Airport_Latitude,Airport_Longitude
0,"Reported a white quad copter at 1,000 FT 4 N o...",4 NM N of NTD,NTD,4,N,7.408,0,34.119274,-119.119585
1,Multiple aircraft observed multiple uas off th...,24 NM SW of DEN,DEN,24,SW,44.448,225,39.861667,-104.673167
2,NaN,2 NM NE of AZO,AZO,2,NE,3.704,45,42.234389,-85.551556


In [14]:
# Merge, drop and rename columns
new_df2 = pd.merge(new_df, airports, on='IDENT', how='left')
new_df2 = new_df2.drop(['Airport_Latitude', 'Airport_Longitude'], 1)
new_df2 = new_df2.rename({'LONGITUDE': 'Airport_Longitude', 'LATITUDE': 'Airport_Latitude'}, axis=1)
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID
0,"Reported a white quad copter at 1,000 FT 4 N o...",4 NM N of NTD,NTD,4,N,7.408,0,Point Mugu NAS (Naval Base Ventura Co),34.119274,-119.119585,KNTD
1,Multiple aircraft observed multiple uas off th...,24 NM SW of DEN,DEN,24,SW,44.448,225,Denver Intl,39.861667,-104.673167,KDEN
2,NaN,2 NM NE of AZO,AZO,2,NE,3.704,45,Kalamazoo/Battle Creek Intl,42.234389,-85.551556,KAZO


In [15]:
# Calculate UAS lat/long
uas_lat = []
uas_long = []

for i in range(len(new_df)):
    lat_airport = pd.to_numeric(new_df2['Airport_Latitude'][i])
    long_airport = pd.to_numeric(new_df2['Airport_Longitude'][i])
    b = pd.to_numeric(new_df['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_airport, long_airport)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append UAS Lat/Long information to DataFrame
new_df2['UAS_Latitude'] = uas_lat
new_df2['UAS_Longitude'] = uas_long
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID,UAS_Latitude,UAS_Longitude
0,"Reported a white quad copter at 1,000 FT 4 N o...",4 NM N of NTD,NTD,4,N,7.408,0,Point Mugu NAS (Naval Base Ventura Co),34.119274,-119.119585,KNTD,34.186058,-119.119585
1,Multiple aircraft observed multiple uas off th...,24 NM SW of DEN,DEN,24,SW,44.448,225,Denver Intl,39.861667,-104.673167,KDEN,39.578015,-105.038978
2,NaN,2 NM NE of AZO,AZO,2,NE,3.704,45,Kalamazoo/Battle Creek Intl,42.234389,-85.551556,KAZO,42.257964,-85.519814


In [16]:
# Export records to csv file
new_df2 = new_df2.drop(['Distance','Bearing','Distance_Kilometers','Bearing_Degrees','ICAO_ID'], 1)
new_df2 = new_df2.rename({'Airport_Longitude': 'Origin_Longitude', 'Airport_Latitude': 'Origin_Latitude'}, axis=1)
new_df2 = pd.merge(new_df2, nonstd, left_on=['REMARKS', 'UAS_LOC'], right_on = ['REMARKS','UASLOCATION'] , how='left')
new_df2.to_csv('COMPLETED-Non-std-3Airports.csv', index = False)

In [ ]:
########################################################################################################

In [17]:
# Find records that reference NAVAID
nonstd_nav = nonstd[(nonstd['DATASET'] == 'NAVAID')]
nonstd_nav

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,LONGITUDE,LATITUDE,UASLOCATION,DATASET
1,PIC reported a white uas pass 500 feet belowwh...,6/10/2019,1:30,2019-06-10T01:30Z,N523GC,DEN,UAS,NaN,ABE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9 NM E of ETX,NAVAID


In [18]:
# Extract IDENT
uas_loc = nonstd_nav['UASLOCATION'].tolist()
uas_navaid = []

for i in range(len(uas_loc)):
    navaid = uas_loc[i].split(' ')[-1]
    
    if len(airport) <= 4:
        uas_navaid.append(navaid)
    else:
        trim_air = navaid[-3:]
        uas_navaid.append(trim_air)
        
uas_navaid_df = pd.DataFrame()
uas_navaid_df['IDENT'] = uas_navaid
uas_navaid_df

,IDENT
0,ETX


In [19]:
# Read in NAVAID dataset
navaid = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/NAVAID_Cleaned.csv')

In [20]:
# Merge IDent with NAVAID dataset
join_uas_nav = pd.merge(uas_navaid_df, navaid, on='IDENT', how='left')
join_uas_nav

,IDENT,LONGITUDE,LATITUDE,NAME_TXT
0,ETX,-75.684034,40.581044,EAST TEXAS


In [21]:
# Append needed columns
add_col = nonstd_nav['REMARKS'].tolist()
join_uas_nav['REMARKS'] = add_col
add_col1 = nonstd_nav['UASLOCATION'].tolist()
join_uas_nav['UASLOCATION'] = add_col1
join_uas_nav

,IDENT,LONGITUDE,LATITUDE,NAME_TXT,REMARKS,UASLOCATION
0,ETX,-75.684034,40.581044,EAST TEXAS,PIC reported a white uas pass 500 feet belowwh...,9 NM E of ETX


In [22]:
# Separate distance and bearing into two columns
uas_loc = join_uas_nav['UASLOCATION']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(float(distance_nm))
    bearings.append(bearing_abrev)
    

# Create DataFrame to store UAS Location Information
new_df = pd.DataFrame(join_uas_nav['REMARKS'])
new_df['UASLOCATION'] = join_uas_nav['UASLOCATION']
new_df['IDENT'] = join_uas_nav['IDENT']
new_df['Distance'] = distances
new_df['Bearing'] = bearings
new_df['NAME_TXT'] = join_uas_nav['NAME_TXT']

new_df

,REMARKS,UASLOCATION,IDENT,Distance,Bearing,NAME_TXT
0,PIC reported a white uas pass 500 feet belowwh...,9 NM E of ETX,ETX,9.0,E,EAST TEXAS


In [23]:
# Convert distance to NM and bearing to degrees
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

bearDegree = pd.DataFrame(new_df['Bearing'])
bearDegree = bearDegree.replace({'Bearing': bearing_deg})

new_df['Distance_Kilometers'] = dist_kilo
new_df['Bearing_Degrees'] = bearDegree
new_df['NAVAID_Latitude'] = join_uas_nav['LATITUDE']
new_df['NAVAID_Longitude'] = join_uas_nav['LONGITUDE']
new_df['NAME_TXT'] = join_uas_nav['NAME_TXT']
new_df

,REMARKS,UASLOCATION,IDENT,Distance,Bearing,NAME_TXT,Distance_Kilometers,Bearing_Degrees,NAVAID_Latitude,NAVAID_Longitude
0,PIC reported a white uas pass 500 feet belowwh...,9 NM E of ETX,ETX,9.0,E,EAST TEXAS,16.668,90,40.581044,-75.684034


In [24]:
# Calculate UAS lat/long
uas_lat = []
uas_long = []


for i in range(len(new_df)):
    lat_nav = pd.to_numeric(new_df['NAVAID_Latitude'][i])
    long_nav = pd.to_numeric(new_df['NAVAID_Longitude'][i])
    b = pd.to_numeric(new_df['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_nav, long_nav)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)
    
# Append UAS Lat/Long information to DataFrame
new_df['UAS_Latitude'] = uas_lat
new_df['UAS_Longitude'] = uas_long


new_df

,REMARKS,UASLOCATION,IDENT,Distance,Bearing,NAME_TXT,Distance_Kilometers,Bearing_Degrees,NAVAID_Latitude,NAVAID_Longitude,UAS_Latitude,UAS_Longitude
0,PIC reported a white uas pass 500 feet belowwh...,9 NM E of ETX,ETX,9.0,E,EAST TEXAS,16.668,90,40.581044,-75.684034,40.580877,-75.487166


In [25]:
# Export record to csv file
new_df = new_df.drop(['Distance','Bearing','Distance_Kilometers','Bearing_Degrees'], 1)
new_df = new_df.rename({'NAVAID_Longitude': 'Origin_Longitude', 'NAVAID_Latitude': 'Origin_Latitude', 'NAME_TXT': 'NAME'}, axis=1)
new_df = pd.merge(new_df, nonstd, on='REMARKS', how='left')
new_df.to_csv('COMPLETED-Non-std-1NNAVAID.csv', index = False)

In [ ]:
################################################################################################################

In [26]:
# Find the remaining 2 designated point records
nonstd_des = nonstd[(nonstd['DATASET'] == 'DESIGNATED POINT')]
nonstd_des

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,LONGITUDE,LATITUDE,UASLOCATION,DATASET
2,LUF airspace manager reported to ZAB aircraft ...,12/14/2020,23:04,2020-12-14T23:04Z,ROOK1,DEN,UAS,NOTIFICATION-NONE,ZAB,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10 NM NW of TANKZ,DESIGNATED POINT
3,LUF airspace manager reported to ZAB aircraft ...,12/14/2020,23:55,2020-12-14T23:55Z,DEADLY1,DEN,UAS,NOTIFICATION-NONE,ZAB,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13 NM SE of BUGGS,DESIGNATED POINT


In [27]:
# Export recrods to csv file and send back to Lex. These records do not reference designated points in fact.
nonstd_des.to_csv('Olga - Additional_Non-Standard_Records_7--2NotDesidnated.csv', index = False)

In [ ]:
###################################################################################################################

In [27]:
# Find record with the Lat/Long calculated in CEDAR
nonstd_done = nonstd[(nonstd['DATASET'] == 'LAT/LONG')]
nonstd_done

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,LONGITUDE,LATITUDE,UASLOCATION,DATASET
5,NaN,7/18/2021,20:30,2021-07-18T20:30Z,LEGCY11,ATO,UAS,NaN,GSO,750990909.0,...,120.0,NaN,79.5702,35.5443,"LEGCY11, 2/V22 GSO overflight reported to RDU ...","Facility summary, weather and falcon bookmark ...",NaN,NaN,NaN,LAT/LONG


In [28]:
# Rename columns for concatenating purpose
nonstd_done = nonstd_done.rename({'UASLONG.CEDAR': 'UAS_Longitude', 'UASLAT.CEDAR': 'UAS_Latitude'}, axis=1)
nonstd_done

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UAS_Longitude,UAS_Latitude,SUMMARY.CEDAR,QAFINDINGS.CEDAR,LONGITUDE,LATITUDE,UASLOCATION,DATASET
5,NaN,7/18/2021,20:30,2021-07-18T20:30Z,LEGCY11,ATO,UAS,NaN,GSO,750990909.0,...,120.0,NaN,79.5702,35.5443,"LEGCY11, 2/V22 GSO overflight reported to RDU ...","Facility summary, weather and falcon bookmark ...",NaN,NaN,NaN,LAT/LONG


In [29]:
# Export to csv file
nonstd_done.to_csv('COMPLETED-Non-std-1LAT-LONG.csv', index = False)

In [ ]:
#####################################################################################################################

In [30]:
# Read in csv file with the record that has a fixed IDENT
nonstd_air = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/Olga - Non-Standard_1OUTOF25 -- RETURNED.csv')
nonstd_air

,IDENT,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,...,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,DATASET,UASLOCATION
0,ALT,FBI detected a uas at the JFK 300002.5 ALT 11...,9/23/2019,18:47,2019-09-23T18:47Z,NaN,JATOC,UAS,TOI-UAS-TFR,JATOC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FIX RADIAL,2.5 NM 300 of JFK


In [31]:
# Extract IDENT
uas_loc = nonstd_air['UASLOCATION'].tolist()
uas_airport = []

for i in range(len(uas_loc)):
    airport = uas_loc[i].split(' ')[-1]
    
    if len(airport) <= 4:
        uas_airport.append(airport)
    else:
        trim_air = airport[-3:]
        uas_airport.append(trim_air)
        
uas_airport_df = pd.DataFrame()
uas_airport_df['IDENT'] = uas_airport
uas_airport_df

,IDENT
0,JFK


In [32]:
# Read in airports dataset
airports = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/airports_cleaned.csv')

# Merge IDent with airports dataset
uas_air_loc= pd.merge(uas_airport_df, airports, on='IDENT', how='left')
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,JFK,John F Kennedy Intl,40.639928,-73.778693,KJFK


In [33]:
# Append needed columns
add_col = nonstd_air['REMARKS'].tolist()
uas_air_loc['REMARKS'] = add_col
add_col1 = nonstd_air['UASLOCATION'].tolist()
uas_air_loc['UASLOCATION'] = add_col1
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,REMARKS,UASLOCATION
0,JFK,John F Kennedy Intl,40.639928,-73.778693,KJFK,FBI detected a uas at the JFK 300002.5 ALT 11...,2.5 NM 300 of JFK


In [34]:
# Separate distnace and bearing into two columns
uas_loc = uas_air_loc['UASLOCATION']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(distance_nm[0:])
    bearings.append(bearing_abrev)

# Create DataFrame to store UAS Location Information
new_df = pd.DataFrame(uas_air_loc['REMARKS'])
new_df['UAS_LOC'] = uas_air_loc['UASLOCATION']
new_df['IDENT'] = uas_air_loc['IDENT']
new_df['Distance'] = distances
new_df['Bearing'] = bearings

new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing
0,FBI detected a uas at the JFK 300002.5 ALT 11...,2.5 NM 300 of JFK,JFK,2.5,300


In [39]:
# Convert distnace to KM
new_df['Distance'] = pd.to_numeric(new_df['Distance'],errors = 'coerce')
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

new_df['Distance_Kilometers'] = dist_kilo
new_df['Bearing_Degrees'] = new_df['Bearing']
new_df['Airport_Latitude'] = uas_air_loc['LATITUDE']
new_df['Airport_Longitude'] = uas_air_loc['LONGITUDE']
new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,Airport_Latitude,Airport_Longitude
0,FBI detected a uas at the JFK 300002.5 ALT 11...,2.5 NM 300 of JFK,JFK,2.5,300,4.63,300,40.639928,-73.778693


In [40]:
# Merge, drop and rename columns
new_df2 = pd.merge(new_df, airports, on='IDENT', how='left')
new_df2 = new_df2.drop(['Airport_Latitude', 'Airport_Longitude'], 1)
new_df2 = new_df2.rename({'LONGITUDE': 'Airport_Longitude', 'LATITUDE': 'Airport_Latitude'}, axis=1)
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID
0,FBI detected a uas at the JFK 300002.5 ALT 11...,2.5 NM 300 of JFK,JFK,2.5,300,4.63,300,John F Kennedy Intl,40.639928,-73.778693,KJFK


In [41]:
# Calculate UAS lat/long
uas_lat = []
uas_long = []

for i in range(len(new_df)):
    lat_airport = pd.to_numeric(new_df2['Airport_Latitude'][i])
    long_airport = pd.to_numeric(new_df2['Airport_Longitude'][i])
    b = pd.to_numeric(new_df['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_airport, long_airport)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append UAS Lat/Long information to DataFrame
new_df2['UAS_Latitude'] = uas_lat
new_df2['UAS_Longitude'] = uas_long
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID,UAS_Latitude,UAS_Longitude
0,FBI detected a uas at the JFK 300002.5 ALT 11...,2.5 NM 300 of JFK,JFK,2.5,300,4.63,300,John F Kennedy Intl,40.639928,-73.778693,KJFK,40.660765,-73.826108


In [42]:
# Export record to csv file
new_df2 = new_df2.drop(['Distance','Bearing','Distance_Kilometers','Bearing_Degrees','ICAO_ID'], 1)
new_df2 = new_df2.rename({'Airport_Longitude': 'Origin_Longitude', 'Airport_Latitude': 'Origin_Latitude'}, axis=1)
new_df2 = pd.merge(new_df2, nonstd_air, on='REMARKS', how='left')
new_df2.to_csv('COMPLETED-Non-std-1NAirports.csv', index = False)

In [ ]:
###################################################################################################################

In [43]:
# Read in the csv file that contains the record with calculated lat/long in CEDAR
lat_long = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/Olga - All_Points_Non-Standard_Runways_1OUTOF425 - cedar.csv')
lat_long

,REMARKS,RUNWAY,DIRECTION,AIRPORT,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,...,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION,DATASET,cityname
0,Aircraft reported nearly colliding with a red ...,['rwy31 '],[],"['DEN', 'PAO', 'CIC', 'CIC', 'DEN', 'AWO']",8/7/2021,21:55,2021-08-07T21:55Z,N879HZ,DEN,UAS,...,130,NaN,-122.12,37.45,"The pilot of N879HZ, a Cessna Skyhawk, reporte...",Reviewed MOR Summary and Falcon for a UAS repo...,RWY 31,NaN,LAT/LONG,NaN


In [44]:
# Rename columns
lat_long = lat_long.rename({'UASLONG.CEDAR': 'UAS_Longitude', 'UASLAT.CEDAR': 'UAS_Latitude'}, axis=1)
lat_long

,REMARKS,RUNWAY,DIRECTION,AIRPORT,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,...,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UAS_Longitude,UAS_Latitude,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION,DATASET,cityname
0,Aircraft reported nearly colliding with a red ...,['rwy31 '],[],"['DEN', 'PAO', 'CIC', 'CIC', 'DEN', 'AWO']",8/7/2021,21:55,2021-08-07T21:55Z,N879HZ,DEN,UAS,...,130,NaN,-122.12,37.45,"The pilot of N879HZ, a Cessna Skyhawk, reporte...",Reviewed MOR Summary and Falcon for a UAS repo...,RWY 31,NaN,LAT/LONG,NaN


In [45]:
# Export record to csv file
lat_long.to_csv('COMPLETED-Non-std-1NLAT-LONG.csv', index = False)

In [ ]:
####################################################################################################################

In [46]:
# Read in the csv file that contains 3 records were removed initially
nonstd2 = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/Olga - All_Points_Non-Standard_Initially_Removed_3OUTOF8 (Initial Dupes).csv')
nonstd2

,REMARKS,UAS.Location,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,...,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,UASLOCATION,DATASET,RWYLOCATION,cityname
0,Reported observing a silver uas operating at 4...,UNKN,4/19/2018,6:17,2018-04-19T06:17Z,JBU454,DEN,UAS,NaN,N90,...,NaN,NaN,NaN,NaN,NaN,NaN,1.5 NM N,CITY,NaN,Farmingdale
1,Reported taking evasive action after encounter...,UNKN,6/26/2019,20:20,2019-06-26T20:20Z,EJA792,DEN,UAS,NaN,N90,...,NaN,NaN,NaN,NaN,NaN,NaN,5 NM SW of LGA,AIRPORT,NaN,NaN
2,PIC observed a blue and white uas with orange ...,UNKN,7/3/2018,19:06,2018-07-03T19:06Z,UAL641,DEN,UAS,NaN,A90,...,NaN,NaN,NaN,NaN,NaN,NaN,10 NM NE,CITY,NaN,Boston


In [47]:
# Find the record that references an airport
nonstd1 = nonstd2[(nonstd2['DATASET'] == 'AIRPORT')]
nonstd1

,REMARKS,UAS.Location,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,...,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,UASLOCATION,DATASET,RWYLOCATION,cityname
1,Reported taking evasive action after encounter...,UNKN,6/26/2019,20:20,2019-06-26T20:20Z,EJA792,DEN,UAS,NaN,N90,...,NaN,NaN,NaN,NaN,NaN,NaN,5 NM SW of LGA,AIRPORT,NaN,NaN


In [48]:
# Extract Ident
uas_loc = nonstd1['UASLOCATION'].tolist()
uas_airport = []

for i in range(len(uas_loc)):
    airport = uas_loc[i].split(' ')[-1]
    
    if len(airport) <= 4:
        uas_airport.append(airport)
    else:
        trim_air = airport[-3:]
        uas_airport.append(trim_air)
        
uas_airport_df = pd.DataFrame()
uas_airport_df['IDENT'] = uas_airport
uas_airport_df

,IDENT
0,LGA


In [49]:
# Merge ident with the airports dataset
uas_air_loc= pd.merge(uas_airport_df, airports, on='IDENT', how='left')
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,LGA,Laguardia,40.77725,-73.872611,KLGA


In [50]:
# Append needed columns
add_col = nonstd1['REMARKS'].tolist()
uas_air_loc['REMARKS'] = add_col
add_col1 = nonstd1['UASLOCATION'].tolist()
uas_air_loc['UASLOCATION'] = add_col1
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,REMARKS,UASLOCATION
0,LGA,Laguardia,40.77725,-73.872611,KLGA,Reported taking evasive action after encounter...,5 NM SW of LGA


In [51]:
# Creat dictionary for bearings and their abbrevaitions
bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
}

# Separate distance and bearing into two columns
uas_loc = uas_air_loc['UASLOCATION']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(distance_nm[0:])
    bearings.append(bearing_abrev)

# Create DataFrame to store UAS Location Information
new_df = pd.DataFrame(uas_air_loc['REMARKS'])
new_df['UAS_LOC'] = uas_air_loc['UASLOCATION']
new_df['IDENT'] = uas_air_loc['IDENT']
new_df['Distance'] = distances
new_df['Bearing'] = bearings

new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing
0,Reported taking evasive action after encounter...,5 NM SW of LGA,LGA,5,SW


In [52]:
# Convert distance to KM and bearing to degrees
new_df['Distance'] = pd.to_numeric(new_df['Distance'],errors = 'coerce')
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

bearDegree = pd.DataFrame(new_df['Bearing'])
bearDegree = bearDegree.replace({'Bearing': bearing_deg})

new_df['Distance_Kilometers'] = dist_kilo
new_df['Bearing_Degrees'] = bearDegree
new_df['Airport_Latitude'] = uas_air_loc['LATITUDE']
new_df['Airport_Longitude'] = uas_air_loc['LONGITUDE']
new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,Airport_Latitude,Airport_Longitude
0,Reported taking evasive action after encounter...,5 NM SW of LGA,LGA,5,SW,9.26,225,40.77725,-73.872611


In [53]:
# Merge, drop and rename columns
new_df2 = pd.merge(new_df, airports, on='IDENT', how='left')
new_df2 = new_df2.drop(['Airport_Latitude', 'Airport_Longitude'], 1)
new_df2 = new_df2.rename({'LONGITUDE': 'Airport_Longitude', 'LATITUDE': 'Airport_Latitude'}, axis=1)
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID
0,Reported taking evasive action after encounter...,5 NM SW of LGA,LGA,5,SW,9.26,225,Laguardia,40.77725,-73.872611,KLGA


In [54]:
# Calculate UAS lat/long
uas_lat = []
uas_long = []

for i in range(len(new_df)):
    lat_airport = pd.to_numeric(new_df2['Airport_Latitude'][i])
    long_airport = pd.to_numeric(new_df2['Airport_Longitude'][i])
    b = pd.to_numeric(new_df['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_airport, long_airport)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append UAS Lat/Long information to DataFrame
new_df2['UAS_Latitude'] = uas_lat
new_df2['UAS_Longitude'] = uas_long
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID,UAS_Latitude,UAS_Longitude
0,Reported taking evasive action after encounter...,5 NM SW of LGA,LGA,5,SW,9.26,225,Laguardia,40.77725,-73.872611,KLGA,40.718261,-73.950107


In [55]:
# Export record to csv file
new_df2 = new_df2.drop(['Distance','Bearing','Distance_Kilometers','Bearing_Degrees','ICAO_ID'], 1)
new_df2 = new_df2.rename({'Airport_Longitude': 'Origin_Longitude', 'Airport_Latitude': 'Origin_Latitude'}, axis=1)
new_df2 = pd.merge(new_df2, nonstd1, on='REMARKS', how='left')
new_df2.to_csv('COMPLETED-Non-std-1OUT3Airports.csv', index = False)

In [ ]:
###################################################################################################################

In [56]:
# Find records that reference a city name
nonstd_city = nonstd2[(nonstd2['DATASET'] == 'CITY')]
nonstd_city

,REMARKS,UAS.Location,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,...,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,UASLOCATION,DATASET,RWYLOCATION,cityname
0,Reported observing a silver uas operating at 4...,UNKN,4/19/2018,6:17,2018-04-19T06:17Z,JBU454,DEN,UAS,NaN,N90,...,NaN,NaN,NaN,NaN,NaN,NaN,1.5 NM N,CITY,NaN,Farmingdale
2,PIC observed a blue and white uas with orange ...,UNKN,7/3/2018,19:06,2018-07-03T19:06Z,UAL641,DEN,UAS,NaN,A90,...,NaN,NaN,NaN,NaN,NaN,NaN,10 NM NE,CITY,NaN,Boston


In [57]:
# Read in NAVAID dataset and set NAME_TXT field to lower case for mergining purpose
navaid = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/NAVAID_Cleaned.csv')
navaid['NAME_TXT'] = navaid['NAME_TXT'].str.lower()
navaid

,LONGITUDE,LATITUDE,IDENT,NAME_TXT
0,-176.676024,51.871896,ADK,mount moffett
1,-151.744257,68.136626,AKP,anaktuvuk pass
2,-149.960164,61.167868,TED,anchorage
3,-149.769204,61.255004,EDF,elmendorf
4,-159.597931,61.590169,ANI,aniak
...,...,...,...,...
1669,-156.788152,71.273426,BRW,barrow
1670,-156.781302,71.282300,VIR,browerville
1671,-75.164524,43.026523,UCA,utica
1672,-90.434470,40.937351,GBG,galesburg


In [58]:
# Rename cityname column to NAME_TXT to match the NAVAID's column
cityname_df = pd.DataFrame()
cityname_df['NAME_TXT'] = nonstd_city['cityname'].tolist()
cityname_df['NAME_TXT'] = cityname_df['NAME_TXT'].str.lower()
cityname_df

,NAME_TXT
0,farmingdale
1,boston


In [63]:
# Append needed columns
add_col = nonstd_city['REMARKS'].tolist()
cityname_df['REMARKS'] = add_col
add_col1 = nonstd_city['UASLOCATION'].tolist()
cityname_df['UASLOCATION'] = add_col1
cityname_df

,NAME_TXT,REMARKS,UASLOCATION
0,farmingdale,Reported observing a silver uas operating at 4...,1.5 NM N
1,boston,PIC observed a blue and white uas with orange ...,10 NM NE


In [64]:
# Join dataframe with NAVAID using the NAME_TXT field
join_uas_nav1 = pd.merge(cityname_df, navaid, on='NAME_TXT', how='left',indicator=True)
join_uas_nav1

,NAME_TXT,REMARKS,UASLOCATION,LONGITUDE,LATITUDE,IDENT,_merge
0,farmingdale,Reported observing a silver uas operating at 4...,1.5 NM N,NaN,NaN,NaN,left_only
1,boston,PIC observed a blue and white uas with orange ...,10 NM NE,-70.989547,42.35746,BOS,both


In [65]:
# Only one record was linked successfully
join_uas_nav = join_uas_nav1[join_uas_nav1['LONGITUDE'].notnull()]
join_uas_nav

,NAME_TXT,REMARKS,UASLOCATION,LONGITUDE,LATITUDE,IDENT,_merge
1,boston,PIC observed a blue and white uas with orange ...,10 NM NE,-70.989547,42.35746,BOS,both


In [66]:
join_uas_nav.to_csv('Non-std-1NAV.csv', index = False)

join_uas_nav = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/Non-std-1NAV.csv')

In [67]:
# Separate distance and bearing into two columns
uas_loc = join_uas_nav['UASLOCATION']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(float(distance_nm))
    bearings.append(bearing_abrev)
    

# Create DataFrame to store UAS Location Information
new_df = pd.DataFrame(join_uas_nav['REMARKS'])
new_df['UASLOCATION'] = join_uas_nav['UASLOCATION']
new_df['IDENT'] = join_uas_nav['IDENT']
new_df['Distance'] = distances
new_df['Bearing'] = bearings
new_df['NAME_TXT'] = join_uas_nav['NAME_TXT']

new_df

,REMARKS,UASLOCATION,IDENT,Distance,Bearing,NAME_TXT
0,PIC observed a blue and white uas with orange ...,10 NM NE,BOS,10.0,NE,boston


In [68]:
# Convert distance to KM and bearing to degrees
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

bearDegree = pd.DataFrame(new_df['Bearing'])
bearDegree = bearDegree.replace({'Bearing': bearing_deg})

new_df['Distance_Kilometers'] = dist_kilo
new_df['Bearing_Degrees'] = bearDegree
new_df['NAVAID_Latitude'] = join_uas_nav['LATITUDE']
new_df['NAVAID_Longitude'] = join_uas_nav['LONGITUDE']
new_df['NAME_TXT'] = join_uas_nav['NAME_TXT']
new_df

,REMARKS,UASLOCATION,IDENT,Distance,Bearing,NAME_TXT,Distance_Kilometers,Bearing_Degrees,NAVAID_Latitude,NAVAID_Longitude
0,PIC observed a blue and white uas with orange ...,10 NM NE,BOS,10.0,NE,boston,18.52,45,42.35746,-70.989547


In [69]:
# Calculate UAS lat/long
uas_lat = []
uas_long = []


for i in range(len(new_df)):
    lat_nav = pd.to_numeric(new_df['NAVAID_Latitude'][i])
    long_nav = pd.to_numeric(new_df['NAVAID_Longitude'][i])
    b = pd.to_numeric(new_df['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_nav, long_nav)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)
    
# Append UAS Lat/Long information to DataFrame
new_df['UAS_Latitude'] = uas_lat
new_df['UAS_Longitude'] = uas_long

new_df


,REMARKS,UASLOCATION,IDENT,Distance,Bearing,NAME_TXT,Distance_Kilometers,Bearing_Degrees,NAVAID_Latitude,NAVAID_Longitude,UAS_Latitude,UAS_Longitude
0,PIC observed a blue and white uas with orange ...,10 NM NE,BOS,10.0,NE,boston,18.52,45,42.35746,-70.989547,42.475242,-70.830294


In [70]:
# Export record to csv file
new_df = new_df.drop(['Distance','Bearing','Distance_Kilometers','Bearing_Degrees'], 1)
new_df = new_df.rename({'NAVAID_Longitude': 'Origin_Longitude', 'NAVAID_Latitude': 'Origin_Latitude', 'NAME_TXT': 'NAME'}, axis=1)
new_df = pd.merge(new_df, nonstd_city, on='REMARKS', how='left')
new_df.to_csv('COMPLETED-Non-std-1Cityname.csv', index = False)

In [ ]:
######################################################################################################################

In [71]:
# Find the remaining record where the city name is Farmingdale 
null = nonstd2[nonstd2['cityname'] == 'Farmingdale']
null

,REMARKS,UAS.Location,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,...,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,UASLOCATION,DATASET,RWYLOCATION,cityname
0,Reported observing a silver uas operating at 4...,UNKN,4/19/2018,6:17,2018-04-19T06:17Z,JBU454,DEN,UAS,NaN,N90,...,NaN,NaN,NaN,NaN,NaN,NaN,1.5 NM N,CITY,NaN,Farmingdale


In [62]:
# Export record to csv file and send back to Lex
new_df.to_csv('Olga - All_Points_Non-Standard_Initially_Removed_3OUTOF8 (Initial Dupes) -- Not_Cityname.csv', index = False)